In [0]:
from pyspark.sql.types import (
    StructType,
    StructField,
    LongType,
    IntegerType,
    StringType,
    DecimalType,
    DateType,
    TimestampType,
    DoubleType,
    DataType
)
from pyspark.sql.functions import (
    udf,
    to_date,
    col,
    to_timestamp,
    regexp_extract,
    regexp_replace,
    isnan,
    count,
    when,
    trim,
    initcap,length,lower,concat_ws,lpad,split
)
from pyspark.sql import SparkSession, DataFrame, Row
import re
import pandas as pd
from pyspark.sql.utils import AnalysisException
# Create a SparkSession
spark = SparkSession.builder.appName("common_functions").getOrCreate()

In [0]:
# common function to format the dataframe columns
def format_column_names(df: DataFrame) -> DataFrame:
    new_column_names = [col.replace(" ", "_").replace("-", "_") for col in df.columns]
    for old_name, new_name in zip(df.columns, new_column_names):
        df = df.withColumnRenamed(old_name, new_name)
    return df



In [0]:
# common Function to extract the price values and eliminate any special characters
def orders_extract_double(
    df: DataFrame, price_col: str, new_col: str = "price"
) -> DataFrame:
    # Remove commas and extract values using regex
    cleaned_col = regexp_replace(col(price_col), ",", "")  # remove commas
    # Extract number pattern (e.g., 1234.56 or 1234)
    double_extracted = regexp_extract(cleaned_col, r"(\d+\.\d+|\d+)", 0)

    return df.withColumn(
        new_col,
        when(
            trim(col(price_col)).rlike(r"\d"), double_extracted.cast("double")
        ).otherwise(None),
    )

In [0]:
# common function to cast columns
def cast_columns(
    df: DataFrame, cast_map: dict[str, DataType], date_format_map: dict[str, str] = {}
) -> DataFrame:
    """
    Cast specified columns of orders DataFrame to given PySpark data types, with optional date formatting.

    Args:
        df (DataFrame): The input DataFrame.
        cast_map (dict): A mapping of column names to PySpark data types (e.g., IntegerType()).
        date_format_map (dict): Optional mapping of date columns to their formats (used only for DateType).

    Returns:
        DataFrame: The DataFrame with casted columns.
    """
    for col_name, data_type in cast_map.items():
        if col_name in df.columns:
            if isinstance(data_type, DateType):
                # fmt = date_format_map.get(col_name, "yyyy-MM-dd")  # default format
                fmt = date_format_map.get(col_name, "dd/MM/yyyy")
                # df = df.withColumn(col_name, to_date(col(col_name), fmt))
                df = df.withColumn(
                    col_name,
                    to_date(
                        concat_ws(
                            "-",
                            lpad(split(col(col_name), "/")[0], 2, "0"),
                            lpad(split(col(col_name), "/")[1], 2, "0"),
                            split(col(col_name), "/")[2],
                        ),
                        "dd-MM-yyyy",
                    ),
                )
            else:
                df = df.withColumn(col_name, col(col_name).cast(data_type))
    return df

In [0]:
# generic function to clean names
def clean_name(df: DataFrame, column_name: str = "name") -> DataFrame:
    """
    Clean the specified string column in a DataFrame.

    Steps:
    - Trim whitespace
    - Remove extra spaces
    - Remove non-alphabetic characters (optional)
    - Capitalize first letters

    Args:
        df (DataFrame): Input DataFrame
        column_name (str): Name of the column to clean

    Returns:
        DataFrame: DataFrame with cleaned name column
    """
    return df.withColumn(
        column_name,
        initcap(
            regexp_replace(
                regexp_replace(
                    trim(col(column_name)), r"\s+", " "
                ),  # multiple spaces to one
                r"[^a-zA-Z\s]",
                "",  # remove non-alphabet characters
            )
        ),
    )

In [0]:
# generic function to clean phone numbers
def clean_phone_numbers(df: DataFrame, column_name: str = "phone") -> DataFrame:
    """
    Cleans phone numbers by:
    - Removing extensions after 'x'
    - Removing all non-digit characters
    - Removing leading '001' or leading zeros
    - Validating only 10-digit numbers

    Args:
        df (DataFrame): Input DataFrame
        column_name (str): The phone number column to clean

    Returns:
        DataFrame: Cleaned phone numbers
    """

    df_cleaned = (
        df
        # Remove anything after x or X
        .withColumn(column_name, regexp_replace(col(column_name), r"[xX].*$", ""))
        # Remove non-digit characters
        .withColumn(column_name, regexp_replace(col(column_name), r"[^\d]", ""))
        # Remove leading '001'
        .withColumn(column_name, regexp_replace(col(column_name), r"^001", ""))
        # Remove leading zeros
        .withColumn(column_name, ltrim(col(column_name)))
        # Keep only valid 10-digit numbers
        .withColumn(
            column_name,
            when(length(col(column_name)) == 10, col(column_name)).otherwise(None),
        )
    )

    return df_cleaned

In [0]:
# generic function to clean email columns
def clean_email_column(df: DataFrame, column_name: str = "email") -> DataFrame:
    """
    Cleans an email column by:
    - Trimming whitespace
    - Converting to lowercase
    - Removing invalid characters
    - Validating the structure of the email

    Args:
        df (DataFrame): Input DataFrame
        column_name (str): The email column to clean

    Returns:
        DataFrame: DataFrame with cleaned emails or null if invalid
    """
    cleaned_df = (
        df.withColumn(column_name, trim(col(column_name))).withColumn(
            column_name, lower(col(column_name))
        )
        # Optional: Remove whitespace and control characters inside email
        .withColumn(column_name, regexp_replace(col(column_name), r"[^\x20-\x7E]", ""))
        # Validate email format using regex
        .withColumn(
            column_name,
            when(
                col(column_name).rlike(
                    r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
                ),
                col(column_name),
            ).otherwise(None),
        )
    )
    return cleaned_df

In [0]:
# generic function to create delta table


def create_and_verify_delta_table(
    spark: SparkSession,
    df: DataFrame,
    catalog_name: str,
    table_name: str,
    database: str = "default",
    mode: str = "overwrite",
    partition_cols: list = None,
    drop_if_exists: bool = False,
    merge_schema: bool = True,
) -> bool:
    """
    Create a Delta table with optional schema evolution and drop existing table.

    Args:
        spark (SparkSession): Spark session.
        df (DataFrame): Input DataFrame.
        table_name (str): Table name (no db prefix).
        database (str): Database name.
        mode (str): Save mode.
        partition_cols (list): List of partition columns.
        drop_if_exists (bool): Drop existing table before create.
        merge_schema (bool): Enable schema merging.

    Returns:
        bool: True if table created and verified; else raises Exception.
    """
    full_table_name = f"{catalog_name}.{database}.{table_name}"
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {database}")

    if drop_if_exists:
        try:
            spark.sql(f"DROP TABLE IF EXISTS {full_table_name}")
        except AnalysisException:
            pass  # Table does not exist; ignore

    writer = (
        df.write.format("delta")
        .mode(mode)
        .option("mergeSchema", str(merge_schema).lower())
    )
    if partition_cols:
        writer = writer.partitionBy(*partition_cols)

    writer.saveAsTable(full_table_name)

    # Verify table creation
    tables = spark.catalog.listTables(f"{catalog_name}.{database}")
    if any(t.name == table_name and t.tableType == "MANAGED" for t in tables):
        return True
    else:
        raise RuntimeError(f"Table {full_table_name} not found after creation!")